In [2]:
from typing import Optional
import numpy as np
from supervision import ImageSink
import cv2
import os

class MyImageSink(ImageSink):
    def save_image(self, image: np.ndarray, image_name: Optional[str] = None,quality:int = 70):

        if image_name is None:
            image_name = self.image_name_pattern.format(self.image_count)

        image_path = os.path.join(self.target_dir_path, image_name)
        cv2.imwrite(image_path, image,[cv2.IMWRITE_JPEG_QUALITY, quality])
        self.image_count += 1
        
        
def zoom_center(img, zoom_factor=1.5):

    y_size = img.shape[0]
    x_size = img.shape[1]
    
    # define new boundaries
    x1 = int(0.5*x_size*(1-1/zoom_factor))
    x2 = int(x_size-0.5*x_size*(1-1/zoom_factor))
    y1 = int(0.5*y_size*(1-1/zoom_factor))
    y2 = int(y_size-0.5*y_size*(1-1/zoom_factor))

    # first crop image then scale
    img_cropped = img[y1:y2,x1:x2]
    return cv2.resize(img_cropped, None, fx=zoom_factor, fy=zoom_factor)


In [3]:
import supervision as sv
from tqdm import tqdm

target_dir="../dataset/motogp"
source_video="../video/motogp.mp4"


with MyImageSink(target_dir_path=target_dir,image_name_pattern="image_{:05d}.jpg",overwrite=True) as sink:
    for image in tqdm(sv.get_video_frames_generator(source_path=source_video, stride=60)):
        crop=zoom_center(img=image,zoom_factor=1.4)
        sink.save_image(image=crop)

2619it [01:30, 28.95it/s]
